## INTRODUCTION TO FULLY CONNECTED LAYERS USING TENSORFLOW

### ARCHITECTURE

![FULLY CONNECTED IMAGE](./images/fully-connected-image)

### DROPOUTS

![DROPOUTS](./images/dropout-image)

### STEP 1: IMPORT LIBRARY AND LOAD DATASET

In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### STEP 2: CREATE AN INTERACTIVE SESSION

In [2]:
sess = tf.InteractiveSession()

### STEP 3: DEFINE SHAPE OF OUR DATASET

In [3]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])
keep_prob = tf.placeholder(tf.float32)

### STEP 4: DEFINE HELPER FUNCTIONS

#### 1. FOR WEIGHTS AND BIASES

In [4]:
W = lambda shape: tf.Variable(tf.truncated_normal(shape, stddev=0.1))
b = lambda shape: tf.Variable(tf.constant(0.1, shape=shape))

### STEP 5: DEFINE MODEL

#### OPTIMISATION 1: 

- Add a dropout layer to each fully connected layer.
- Activate the dropouts only during training by setting `keep_prob<1`
- During testing make the `keep_prob=1` to use all the neurons

#### FULLY CONNECTED LAYER 1

In [5]:
W_fc1 = W(shape=[784, 200])
b_fc1 = b(shape=[200])

fc1 = tf.nn.relu(tf.matmul(x, W_fc1) + b_fc1)
fc1_drop = tf.nn.dropout(fc1, keep_prob=keep_prob)

#### FULLY CONNECTED LAYER 2

In [6]:
W_fc2 = W(shape=[200, 100])
b_fc2 = b(shape=[100])

fc2 = tf.nn.relu(tf.matmul(fc1_drop, W_fc2) + b_fc2)
fc2_drop = tf.nn.dropout(fc2, keep_prob=keep_prob)

#### FULLY CONNECTED LAYER 3

In [7]:
W_fc3 = W(shape=[100, 60])
b_fc3 = b(shape=[60])

fc3 = tf.nn.relu(tf.matmul(fc2_drop, W_fc3) + b_fc3)
fc3_drop = tf.nn.dropout(fc3, keep_prob=keep_prob)

#### FULLY CONNECTED LAYER 4

In [8]:
W_fc4 = W(shape=[60, 30])
b_fc4 = b(shape=[30])

fc4 = tf.nn.relu(tf.matmul(fc3_drop, W_fc4) + b_fc4)
fc4_drop = tf.nn.dropout(fc4, keep_prob=keep_prob)

#### READOUT LAYER

In [9]:
W_fc5 = W(shape=[30, 10])
b_fc5 = b(shape=[10])

y = fc5 = tf.matmul(fc4, W_fc5) + b_fc5

### STEP 6: DEFINE METRICS

#### 1. LOSS IN THE FORM OF CROSS ENTROPY

In [10]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

#### 2. TRAINING USING GRADIENT DESCENT OPTIMIZER

#### Optimisation 2:

- Use an exponential decay learning rate
- This will help us in the long run to smoothen the learning curve
- By enabling us do gather more information at the early stage and at a later stage work with finer details
- Also decreasing the learning rate simply also works

##### FORMULA:  `decayed_learning_rate = learning_rate * decay_rate ^ (global_step / decay_steps)`

In [11]:
global_step = tf.Variable(0, trainable=False)

learning_rate = tf.train.exponential_decay(
    learning_rate=0.5,
    global_step=global_step,
    decay_steps=10000,
    decay_rate=0.96
)

In [12]:
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy, global_step)

#### 3. ACCURACY OF THE MODEL

In [13]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

### STEP 7: TRAIN MODEL

In [14]:
sess.run(tf.global_variables_initializer())

In [15]:
for i in range(1000):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob:0.75})
    
    
    ### TEST OUR MODEL INBETWEEN
    if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={ x: batch[0], y_: batch[1], keep_prob:1 })
        print('STEP', i, ':', 'ACCURACY', train_accuracy) 

STEP 0 : ACCURACY 0.19
STEP 100 : ACCURACY 0.89
STEP 200 : ACCURACY 0.92
STEP 300 : ACCURACY 0.93
STEP 400 : ACCURACY 0.99
STEP 500 : ACCURACY 0.99
STEP 600 : ACCURACY 1.0
STEP 700 : ACCURACY 0.96
STEP 800 : ACCURACY 0.97
STEP 900 : ACCURACY 0.98


### STEP 8: EVALUATE MODEL

In [16]:
final_accuracy = accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob:1})
print('FINAL ACCURACY:', final_accuracy)

FINAL ACCURACY: 0.9574


### STEP 9: TEST PREDICTION

In [17]:
x_actual = mnist.test.images[0:1]
y_actual = mnist.test.labels[0:1]

print('ACTUAL VALUE:', y_actual)

ACTUAL VALUE: [[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]]


In [18]:
y_predicted = sess.run(y, feed_dict={x: x_actual, keep_prob:1})

print('PREDICTED VALUE:', y_predicted)

PREDICTED VALUE: [[ -3.2056613   -0.16570112   3.44758892   2.47242165  -3.50097895
   -1.5888747   -8.80059433  13.48353291  -2.2689414    3.94732261]]


In [19]:
output_actual = sess.run(tf.argmax(y_actual, 1))
output_predicted = sess.run(tf.argmax(y_predicted, 1))

print ('EXPECTED OUTPUT:', output_actual)
print ('PREDICTED OUTPUT:', output_predicted)

EXPECTED OUTPUT: [7]
PREDICTED OUTPUT: [7]
